# Simulation study - Inference of pharmacokinetic model in a virtual patient

## Build one compartment PK model

In [1]:
import erlotinib as erlo

# Define mechanistic model
path = erlo.ModelLibrary().one_compartment_pk_model()
mechanistic_model = erlo.PharmacokineticModel(path)
mechanistic_model.set_administration(compartment='central', direct=False)
mechanistic_model.set_parameter_names(names={
    'central.drug_amount': 'Initial plasma drug amount in mg',
    'dose.drug_amount': 'Initial dose comp. drug amount in mg',
    'central.size': 'Volume of distribution in L',
    'dose.absorption_rate': 'Absorption rate in 1/d',
    'myokit.elimination_rate': 'Elimination rate in 1/d'})

# Define error model
error_model = erlo.ConstantAndMultiplicativeGaussianErrorModel()

# Build model
problem = erlo.ProblemModellingController(
    mechanistic_model, error_model)
problem.fix_parameters(name_value_dict={
    'Initial plasma drug amount in mg': 0,
    'Initial dose comp. drug amount in mg': 0})

## Measure plasma concentration in a virtual patient
Applied dose: daily oral dose of 2 mg, starting from day 3.

In [2]:
import numpy as np

# Get predictive model
predictive_model = problem.get_predictive_model()

# Define a virtual patient
parameters = [
    0.8,   # Volume of distribution in central compartment [L]
    55,    # Absorption rate from dose to central compartment [1/d]
    5,     # Elimination rate from central compartment [1/d]
    0.04,  # Base noise
    0.1]   # Relative noise

# Define dosing regimen
predictive_model.set_dosing_regimen(dose=10, start=0, period=1)

# Take virtual measurements
seed = 42
times = np.linspace(0.2, 2, num=10)
n_samples = 1
data = predictive_model.sample(parameters, times, n_samples, seed, include_regimen=True)

### Visualise virtual measurements

In [3]:
# Create scatter plot
fig = erlo.plots.PKTimeSeriesPlot()
fig.add_data(data, biomarker='central.drug_concentration', meas_key='Sample')
fig.set_axis_labels(
    time_label=r'$\text{Time in day}$', 
    biom_label=r'$\text{Plasma conc. in ng/mL}$', 
    dose_label=r'$\text{Dose in mg}$')

# Show figure
fig.show()

**Figure 1:** Visualisation of the measured tumour growth in a virtual patient.

## Build prior predictive model

In [4]:
import pints

# Define prior distribution
log_priors = [
    pints.HalfCauchyLogPrior(location=0, scale=1),     # Volume of distribution
    pints.HalfCauchyLogPrior(location=1, scale=2),     # Absorption rate
    pints.HalfCauchyLogPrior(location=0, scale=0.1),   # Elimination rate
    pints.HalfCauchyLogPrior(location=0, scale=1),     # Sigma base
    pints.HalfCauchyLogPrior(location=0, scale=0.1)]   # Sigma rel.
log_prior = pints.ComposedLogPrior(*log_priors)

# Define prior predictive model and sample 1000 virtual patients
model = erlo.PriorPredictiveModel(predictive_model, log_prior)
n_samples = 1000
samples = model.sample(times, n_samples, seed)

# Visualise prior predictive model
fig = erlo.plots.PDPredictivePlot()
fig.add_prediction(data=samples, bulk_probs=[0.3, 0.6, 0.9])
fig.set_axis_labels(xlabel=r'$\text{Time in day}$', ylabel=r'$\text{Plasma conc. in ng/mL}$')
fig.show()

**Figure 2:** Prior predictive model of tumour volume measurement over time. The shaded areas estimate the 0.3, 0.6 and 0.9 bulk probability of the prior predictive distribution by sampling 1000 virtual "measurements" at each of the time points.

## Find MAP estimates for model parameters

In [5]:
# Create posterior
problem.set_data(data, meas_key='Sample')
problem.set_log_prior(log_priors)
log_posterior = problem.get_log_posteriors()

# Find maximum a posteriori probability estimates (MAP)
opt = erlo.OptimisationController(log_posterior)
opt.set_transform(transform=pints.LogTransformation(n_parameters=5))
map_estimates = opt.run(show_run_progress_bar=True)

### Visualise optimisation results

In [6]:
fig = erlo.plots.ParameterEstimatePlot()
fig.add_data(map_estimates)

fig.show()

**Figure 3:** Maximum a posteriori (MAP) estimates of the model parameters. The y axis displays the estimated parameter value, and the x axis the corresponding individual.

## Find posterior probability distribution

In [7]:
# Set up sampling controller
sampler = erlo.SamplingController(log_posterior)
sampler.set_initial_parameters(data=map_estimates)
sampler.set_transform(transform=pints.LogTransformation(n_parameters=5))

# Run sampling
posterior_samples = sampler.run(n_iterations=4000, show_progress_bar=True)

### Visualise posterior samples

In [8]:
fig = erlo.plots.MarginalPosteriorPlot()
fig.add_data(data=posterior_samples, warm_up_iter=2000)

fig.show()

**Figure 4:** Marginal posterior distributions of model parameters. The y axis displays the sampled parameter value, and the x axis the binned number of samples for each individual.

## Posterior predictive checks

In [9]:
# Sample from posterior predictive model
times = np.linspace(0, 2)
n_samples = 1000
model = erlo.PosteriorPredictiveModel(predictive_model, posterior_samples)
samples = model.sample(times, n_samples)

# Visualise posterior predictive model
fig = erlo.plots.PDPredictivePlot()
fig.add_prediction(data=samples)
fig.add_data(data, meas_key='Sample')
fig.set_axis_labels(xlabel=r'$\text{Time in day}$', ylabel=r'$\text{Plasma concentration in ng/mL}$')
fig.show()

**Figure 5:** Posterior predictive model of future measurements of the virtual patient (ID 1). The shaded area illustrates the approximate 90% bulk probability of the posterior predictive model constructed from 1000 samples from the posterior distribution and subsequent virtual "measurements" for each of 50 equidistant time points. The circles represent the measurements of the virtual patient that were used to infer the posterior distirbution.

In [11]:
# Sample from predictive model
parameters = [0.94, 12.84, 5.31, 0.04, 0.07]
samples = predictive_model.sample(parameters, times, n_samples)

# Visualise posterior predictive model
fig = erlo.plots.PDPredictivePlot()
fig.add_prediction(data=samples)
fig.add_data(data, biomarker='central.drug_concentration', meas_key='Sample')
fig.set_axis_labels(xlabel=r'$\text{Time in day}$', ylabel=r'$\text{Plasma concentration in ng/mL}$')
fig.show()

**Figure 6:** Predictive model of future measurements of the virtual patient (ID 1) cosntructed from the MAP estimates. The shaded area illustrates the approximate 90% bulk probability of the posterior predictive model constructed from 1000 samples from the posterior distribution and subsequent virtual "measurements" for each of 50 equidistant time points. The circles represent the measurements of the virtual patient that were used to infer the posterior distirbution.

## Predict dose-response

In [15]:
# Define a virtual patient
parameters = [
    0.8,   # Volume of distribution in central compartment [L]
    55,    # Absorption rate from dose to central compartment [1/d]
    5,     # Elimination rate from central compartment [1/d]
    0.04,  # Base noise
    0.1]   # Relative noise

# Define dosing regimen
predictive_model.set_dosing_regimen(dose=2, start=0, period=0.3)

# Take virtual measurements
seed = 42
times = np.linspace(8, 10, num=100)
n_samples = 1
data = predictive_model.sample(parameters, times, n_samples, seed)

In [16]:
# Sample from posterior predictive model
times = np.linspace(8, 10, num=100)
n_samples = 1000
model = erlo.PosteriorPredictiveModel(predictive_model, posterior_samples)
samples = model.sample(times, n_samples)

# Visualise posterior predictive model
fig = erlo.plots.PDPredictivePlot()
fig.add_prediction(data=samples)
fig.add_data(data, meas_key='Sample')
fig.set_axis_labels(xlabel=r'$\text{Time in day}$', ylabel=r'$\text{Plasma concentration in ng/mL}$')
fig.show()

**Figure 7:**

In [17]:
# Predict dose-response with MAP model
parameters = [0.94, 12.84, 5.31, 0.04, 0.07]
samples = predictive_model.sample(parameters, times, n_samples)

# Visualise posterior predictive model
fig = erlo.plots.PDPredictivePlot()
fig.add_prediction(data=samples)
fig.add_data(data, biomarker='central.drug_concentration', meas_key='Sample')
fig.set_axis_labels(xlabel=r'$\text{Time in day}$', ylabel=r'$\text{Plasma concentration in ng/mL}$')
fig.show()

**Figure 8:**